## Тестовое (data engineering)
### Шкуратова Д.В.

## 1. Схема БД

- Users
  - guid uuid, pk 
  - username varchar(100) unique not null
  - date_created timestamp
    

-  Topics
    - guid uuid
    - user_guid uuid 
    - title varchar(500)
    - content text
        
- Messages
    - guid uuid pk 
    - theme_guid uuid ref
    - user_guid uuid ref
    - body text
    
- Logs
    - guid uuid pk
    - user_guid uuid ref
    - log_type_guid uuid ref
    - log_date timestamp
    - server_response int IN(0,1)
    - message_id uuid ref
    - theme_guid uuid ref
  

- log_event
    - guid uuid pk
    - event_name varchar(100)




![logsddl.png](logsddl.png)

## Партицирование таблицы Logs

Если в таблице Logs будет храниться очень большое количество записей для каждого дня, а составлять отчеты нужно, например, только за последние месяцы, то можно настроить секционирование таблицы. Тогда для каждого месяца нужно создавать отдельную таблицу.


Пример секционирования таблицы Logs, с созданной секцией, в которой будут храниться только записи за февраль 2025 года. Секции могут унаследовать только ограничение not null, первичные и внешние ключи, ограничения исключения нужно задавать для каждой секции отдельно. 
```PostgreSQL
create table logs(
    guid uuid,
    log_date timestamp not null,
    user_guid uuid,
    topic_guid uuid,
    message_guid uuid,
    response_status_code int not null ,
    log_message varchar(1024),
    event_guid uuid

) partition by range (log_date);

create table logs_y2025m02 partition of logs
for values from ('2025-02-01') to ('2025-03-01');

alter table logs_y2025m02 add constraint logs_y2025m02_pk primary key (guid);
alter table logs_y2025m02 add constraint user_logs_fk
    foreign key (user_guid) references users (guid) on delete set null;
alter table logs_y2025m02 add constraint topic_logs_fk
    foreign key (topic_guid) references topics (guid) on delete set null;
alter table logs_y2025m02 add constraint message_logs_fk
    foreign key (message_guid) references messages (guid) on delete set null;
alter table logs_y2025m02 add constraint event_logs_fk
    foreign key (event_guid) references log_event (guid);
alter table logs_y2025m02 add constraint valid_status_code check ( response_status_code in (0,1) );
```


## 2. Генерация данных

Generator - принимает на вход дату начала и дату окончания, данные генерируются на каждый день временного отрезка 

script.py  - скрипт заполнения БД сгенерированными данными с помощью библиотеки psycopg2



# 3. Docker

#### Сервисы:
1. **database** - контейнер с БД, расширенный скриптом с ddl запросами
2. **seed_db** - контейнер со скриптом генерации датасета

## 4. Скрипт агрегации данных

На вход **aggregate.py** передается начальная и конечная дата периода расчета в формате YYYY-DD-MM
Результат запроса сохраняется в csv файл, который содержит:
- log_date - дата, округленная до дня
- new_users - количество новых пользователей за день
- anon_pcnt - процент сообщений, написанных анонимами за день
- msg_cnt - общее количество сообщений за день
- new_topic_pcnt - процент изменения количества тем относительно предыдущего дня

In [4]:
from sqlalchemy import create_engine
import pandas as pd

host = 'localhost:5422'
user = 'admin'
database_name = 'logsdb'
password = 'admin'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database_name}')

In [5]:
with open('../Task4/request.sql') as f:
    query = f.read()

pd.read_sql(query, engine)

,log_date,new_users,anon_pcnt,msg_cnt,new_topic_pcnt
0,2025-02-10,20,51,170,0.0
1,2025-02-11,20,50,190,400.0
2,2025-02-12,7,51,208,300.0
3,2025-02-13,5,46,314,30.0
4,2025-02-14,5,44,226,0.0
5,2025-02-15,19,39,38,211.5
6,2025-02-16,8,46,62,28.4
7,2025-02-17,2,46,315,10.6
8,2025-02-18,24,47,250,9.6
9,2025-02-19,16,47,413,-0.8
